## 实验介绍

### 1.实验内容

本实验包括: 
* 基于朴素贝叶斯算法的言论过滤器
* 基于朴素贝叶斯算法实现垃圾邮件过滤

### 2.实验目标

通过本实验掌握朴素贝叶斯算法原理，熟悉朴素贝叶斯算法的简单应用。

### 3.实验知识点

* 朴素贝叶斯算法

### 4.实验环境

* python 3.6.5
* numpy 1.13.3
* matplotlib 2.2.3

### 实验准备

点击屏幕右上方的下载实验数据模块，选择下载bayes_email.tgz到指定目录下，然后再依次选择点击上方的File->Open->Upload,上传刚才下载的数据集压缩包，再使用如下命令解压：

In [98]:
!tar -zxvf bayes_email.tgz

x bayes_email/
x bayes_email/spam/
x bayes_email/spam/1.txt
x bayes_email/spam/10.txt
x bayes_email/spam/11.txt
x bayes_email/spam/12.txt
x bayes_email/spam/13.txt
x bayes_email/spam/14.txt
x bayes_email/spam/15.txt
x bayes_email/spam/16.txt
x bayes_email/spam/17.txt
x bayes_email/spam/18.txt
x bayes_email/spam/19.txt
x bayes_email/spam/2.txt
x bayes_email/spam/20.txt
x bayes_email/spam/21.txt
x bayes_email/spam/22.txt
x bayes_email/spam/23.txt
x bayes_email/spam/24.txt
x bayes_email/spam/25.txt
x bayes_email/spam/3.txt
x bayes_email/spam/4.txt
x bayes_email/spam/5.txt
x bayes_email/spam/6.txt
x bayes_email/spam/7.txt
x bayes_email/spam/8.txt
x bayes_email/spam/9.txt
x bayes_email/ham/
x bayes_email/ham/1.txt
x bayes_email/ham/10.txt
x bayes_email/ham/11.txt
x bayes_email/ham/12.txt
x bayes_email/ham/13.txt
x bayes_email/ham/14.txt
x bayes_email/ham/15.txt
x bayes_email/ham/16.txt
x bayes_email/ham/17.txt
x bayes_email/ham/18.txt
x bayes_email/ham/19.txt
x bayes_email/ham/2.txt
x bayes

In [99]:
# -*- coding: UTF-8 -*-
import re
import os
import numpy as np
import random
import matplotlib.pyplot as plt

## 【言论过滤器】

## 实验步骤：【言论过滤器】- 概述

以在线社区留言为例。为了不影响社区的发展，我们要屏蔽侮辱性的言论，所以要构建一个快速过滤器，如果某条留言使用了负面或者侮辱性的语言，那么就将该留言标志为内容不当。过滤这类内容是一个很常见的需求。对此问题建立两个类型：侮辱类和非侮辱类，使用1和0分别表示。  

## 实验步骤：【言论过滤器】- 词条切分和词性标注

我们把文本看成单词向量或者词条向量，也就是说将句子转换为向量。考虑出现所有文档中的单词，再决定将哪些单词纳入词汇表或者说所要的词汇集合，然后必须要将每一篇文档转换为词汇表上的向量。简单起见，我们先假设已经将本文切分完毕，存放到列表中，并对词汇向量进行分类标注。编写代码如下：

In [100]:
def loadDataSet():
    """
    函数说明:创建实验样本
    Parameters:
        无
    Returns:
        postingList - 实验样本切分的词条
        classVec - 类别标签向量
    """
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec

## 实验步骤：【言论过滤器】- 生成词条向量

继续编写代码，并将切分好的词条转换为词条向量。

In [101]:
def setOfWords2Vec(vocabList, inputSet):
    """
    函数说明: 根据vocabList词汇表，将inputSet向量化，向量的每个元素为1或0
    Parameters:
        vocabList - createVocabList返回的列表
        inputSet - 切分的词条列表
    Returns:
        returnVec - 文档向量,词集模型
    """
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec


def createVocabList(dataSet):
    """
    函数说明: 将切分的实验样本词条整理成不重复的词条列表，也就是词汇表

    Parameters:
        dataSet - 整理的样本数据集
    Returns:
        vocabSet - 返回不重复的词条列表，也就是词汇表
    """
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

## 实验步骤：【言论过滤器】-训练朴素贝叶斯分类器 

通过词条向量训练朴素贝叶斯分类器。

Tips：
* 利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率。如果其中有一个概率值为0，那么最后的乘积也为0。为降低这种影响，可以将所有词的出现数初始化为1， 并将分母初始化为2。这种做法就叫做拉普拉斯平滑(Laplace Smoothing)，是比较常用的平滑方法。

* 下溢问题。由于太多很小的数相乘会导致下溢问题。两个小数相乘，越乘越小，这样就造成了下溢出。在程序中，许多很小的数相乘，最后四舍五入计算结果可能就变成0。为了解决这个问题，对乘积结果取自然对数。通过求对数可以避免下溢出或者浮点数舍入导致的错误。

In [102]:
def trainNB(trainMatrix, trainCategory):
    """
    函数说明:朴素贝叶斯分类器训练函数

    Parameters:
        trainMatrix - 训练文档矩阵，即setOfWords2Vec返回的returnVec构成的矩阵
        trainCategory - 训练类别标签向量，即loadDataSet返回的classVec
    Returns:
        p0Vect - 非的条件概率数组
        p1Vect - 侮辱类的条件概率数组
        pAbusive - 文档属于侮辱类的概率
    """
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)
    p0Denom = 0.0
    p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vec = p1Num / p1Denom
    p0Vec = p0Num / p0Denom
    return p0Vec, p1Vec, pAbusive

## 实验步骤：【言论过滤器】- 使用分类器进行分类

In [103]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    """
    函数说明:朴素贝叶斯分类器分类函数

    Parameters:
        vec2Classify - 待分类的词条数组
        p0Vec - 非侮辱类的条件概率数组
        p1Vec -侮辱类的条件概率数组
        pClass1 - 文档属于侮辱类的概率
    Returns:
        0 - 属于非侮辱类
        1 - 属于侮辱类
    """
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0


def testingNB():
    """
    函数说明:测试朴素贝叶斯分类器
    测试样本1：['love', 'my', 'dalmation']
    测试样本2：['stupid', 'garbage']
    Parameters:
        无
    Returns:
        无
    """
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))

In [104]:
if __name__ == '__main__':
    testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


## 【垃圾邮件过滤】

## 实验步骤：【垃圾邮件过滤】- 概述

使用朴素贝叶斯解决一些现实生活中的问题时，需要先从文本内容得到字符串列表，然后生成词向量。本实验中，我们将了解朴素贝叶斯的一个最著名的应用：电子邮件垃圾过滤。首先看一下使用朴素贝叶斯对电子邮件进行分类的步骤：

* 收集数据：提供文本文件。
* 准备数据：将文本文件解析成词条向量。
* 分析数据：检查词条确保解析的正确性。
* 训练算法：使用我们之前建立的trainNB()函数。
* 测试算法：使用classifyNB()函数，并构建一个新的测试函数来计算文档集的错误率。
* 使用算法：构建一个完整的程序对一组文档进行分类，将错分的文档输出到屏幕上。

数据集路径为：bayes_email，该目录下有两个文件夹：ham和spam，其中spam文件下的txt文件为垃圾邮件。

### 数据示例
![](dataIntro_bayes.png)

## 实验步骤：【垃圾邮件过滤】- 文本切分 

对于英文文本，我们可以以非字母、非数字作为符号进行切分，使用split函数即可。编写代码如下：

In [105]:
def textParse(bigString):
    """
    函数说明:接收一个字符串并将其解析为字符串列表

    Parameters:
        bigString - 字符串
    Returns:
        字符串列表(除了单个字母，例如大写的I，其它单词变成小写，同时过滤长度小于3的字符串)
    """
    import re
    listOfTokens = re.split(r'\w*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

## 实验步骤：【垃圾邮件过滤】- 分类

根据词汇表，将每个文本向量化。将数据集分为训练集和测试集，使用交叉验证的方式测试朴素贝叶斯分类器的准确性。编写代码如下：

In [106]:
def spamTest():
    """
    函数说明: 将数据集分为训练集和测试集，使用交叉验证的方式测试朴素贝叶斯分类器的准确性
    """
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):
        # 导入并解析文本文件
        wordList = textParse(open('bayes_email/spam/%d.txt' % i, encoding='cp1252').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('bayes_email/ham/%d.txt' % i, encoding='cp1252').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)

    vocabList = createVocabList(docList)
    trainingSet = list(range(50))
    testSet = []
    # 随机取10个邮件用来测试
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del (trainingSet[randIndex])

    trainMat = []
    trainClasses = []

    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])

    p0V, p1V, pSpam = trainNB(np.array(trainMat), np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ', float(errorCount) / len(testSet))

In [107]:
if __name__ == '__main__':
    spamTest()

the error rate is:  0.5
